## AWS Sentinel2 COG data extraction using STAC
Extract some data into xarray using STAC, visualize it with Holoviz

In [ ]:
import numpy as np
import xarray as xr
import stackstac
import pystac_client
import hvplot.xarray
import panel as pn

Turn off some annoying warnings:

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import param
param.get_logger().setLevel(param.ERROR)

### Create a Dask cluster

To cut down on the execution time when accessing large amounts of data, we can use a Dask cluster to do the computation in parallel. 

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client()

In [ ]:
client

In [ ]:
#client.close();cluster.shutdown()

Using `pystac_client` we can search STAC endpoints for items matching our query parameters.

#### Explore what collections exist on a STAC API endpoint

In [ ]:
stac_api_endpoint = 'https://earth-search.aws.element84.com/v0'
stac = pystac_client.Client.open(stac_api_endpoint)

In [ ]:
for collection in stac.get_all_collections():
    print(collection)

#### Search for data in collections

In [ ]:
collections=["sentinel-s2-l2a-cogs"]
datetime = "2017-12-01/2018-01-01"
# bbox_lonlat = [40.09, -2.98, 40.61, -2.46]   #africa
bbox_lonlat = [151.2957545, -33.7390216, 151.312234, -33.7012561] # AUS, coastsat
cloud_max = 60

In [ ]:
search = stac.search(
    bbox=bbox_lonlat,
    datetime=datetime,
    collections=collections,
    limit=500,  # fetch items in batches of 500
    query={"eo:cloud_cover": {"lt": cloud_max}},
)

items = list(search.get_items())
print(len(items))

In [ ]:
items_as_dict = [item.to_dict() for item in items]

#### Use StackStac to open the items in xarray

In [ ]:
da = (
    stackstac.stack(
        items_as_dict,
        bounds_latlon = bbox_lonlat,
        assets=["B04", "B03", "B02"],  # red, green, blue
        chunksize=4096,    
        resolution=10,
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
#    .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
)
da

### Visualize with Holoviz 
Holoviz doesn't like all the extra coordinates, so drop everything except 'x','y','time', and 'band'.  And scale the R,G,B data to between [0,1]

In [ ]:
drop_coords = [x for x in list(da.coords) if not x in ['x','y','time','band']]
da = da.drop_vars(drop_coords)

In [ ]:
dmean = float(da.mean())
dstd = float(da.std())
vmin = max(dmean - 2*dstd,0)
vmax = dmean + 2*dstd
print(vmax)

In [ ]:
da2 = da/vmax

In [ ]:
da2.hvplot.rgb(x='x', y='y',  bands='band', crs=32756, rasterize=True, 
                 frame_width=200, widgets={'time': pn.widgets.Select})